In [134]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydot
from keras.utils import plot_model

In [38]:
import h5py
import pickle
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [2]:
import tensorflow as tf
import json
import seaborn as sns
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import  cross_val_score
from sklearn.grid_search import GridSearchCV
from time import time
from operator import itemgetter
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn import metrics
import pickle 
import scipy
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import os
import numpy as np
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV
import gensim
import pickle
from gensim.models.doc2vec import LabeledSentence

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
import keras
from keras.preprocessing import sequence
import math
%matplotlib inline

//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.
//anaconda/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [3]:

from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import multiprocessing
import re
import gensim.models.word2vec
import gensim.models.word2vec as w2v
np.random.seed(13)
from keras.models import Model
from keras.layers import Dense, Embedding, GRU, Input, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.regularizers import l2
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
from matplotlib import pyplot as plt
from keras.utils import plot_model
%matplotlib inline
plt.style.use("ggplot")

In [4]:
with open("big_data.txt", "rb") as fp:   # Unpickling
...   data = pickle.load(fp)

In [5]:
with open("big_data_label.txt", "rb") as fp:   # Unpickling
...   labels = pickle.load(fp)



## OUR DATA

In [52]:
X_train, X_test, y_train, y_test = train_test_split(data,labels, test_size=.3,
                                                  random_state=5)

In [53]:
len(X_train), len(X_test)

(644486, 276209)

In [54]:
len(y_train), len(y_test)

(644486, 276209)

In [55]:
maxlen = 5

In [56]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)

In [57]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_test)
X_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [58]:
X_train.shape

(644486, 5)

In [59]:
X_test.shape

(276209, 5)

In [60]:
y_train = np_utils.to_categorical(y_train, len(set(labels)))
y_test = np_utils.to_categorical(y_test, len(set(labels)))

In [61]:
y_train.shape,y_test.shape

((644486, 5), (276209, 5))

In [66]:
## The words are embedded and passed into a convolutional layer , whose outputs are passed into LSTM 
## We can optimize by trying different parameters 
#embedding_vector_length = 16
model = Sequential()
model.add(Embedding(100, embedding_vector_length))
model.add(Convolution1D(filters=8, kernel_size=5, padding='same', activation='relu',input_shape=(5,1)))
model.add(Convolution1D(filters=4, kernel_size=4, padding='same', activation='relu'))
model.add(Convolution1D(filters=2, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(64, input_dim=64, input_length=10, return_sequences=True))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, return_sequences=True, input_shape=(10, 64))`


In [135]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 5, 8)              48        
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 5, 4)              132       
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 5, 2)              26        
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 2, 2)              0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 2, 64)             17152     
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 165       
Total para

In [70]:
history = model.fit(X_train.reshape(X_train.shape[0],X_train.shape[1],1), y_train,
                    epochs=10, batch_size=128,
                    verbose=1, validation_split=0.3)

Train on 451140 samples, validate on 193346 samples
Epoch 1/10
451140/451140 [==============================] - 93s - loss: 1.0034 - acc: 0.5849 - val_loss: 0.9527 - val_acc: 0.6056
Epoch 2/10
451140/451140 [==============================] - 106s - loss: 0.9345 - acc: 0.6151 - val_loss: 0.9043 - val_acc: 0.6284
Epoch 3/10
451140/451140 [==============================] - 93s - loss: 0.8991 - acc: 0.6291 - val_loss: 0.8897 - val_acc: 0.6321
Epoch 4/10
451140/451140 [==============================] - 93s - loss: 0.8766 - acc: 0.6377 - val_loss: 0.8820 - val_acc: 0.6352
Epoch 5/10
451140/451140 [==============================] - 97s - loss: 0.8622 - acc: 0.6431 - val_loss: 0.8597 - val_acc: 0.6430
Epoch 6/10
451140/451140 [==============================] - 95s - loss: 0.8504 - acc: 0.6467 - val_loss: 0.8314 - val_acc: 0.6538
Epoch 7/10
451140/451140 [==============================] - 97s - loss: 0.8416 - acc: 0.6506 - val_loss: 0.8253 - val_acc: 0.6556
Epoch 8/10
451140/451140 [===========

In [71]:
np.array(X_test).shape

(276209, 5)

In [72]:
score = model.evaluate(X_test.reshape(X_test.shape[0],X_test.shape[1],1), y_test,
                       batch_size=128, verbose=1)

276209/276209 [==============================] - 16s    


In [73]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.814912068212
Test accuracy: 0.6629363996


In [74]:
y_predicted_arr = model.predict(X_test.reshape(X_test.shape[0],X_test.shape[1],1))

In [75]:
X_test.reshape(X_test.shape[0],X_test.shape[1],1).shape

(276209, 5, 1)

In [76]:
y_predicted = []
for i in y_predicted_arr:
    y_predicted.append(np.where(np.max(i) == i)[0][0])

In [77]:
Y_test = []
for i in y_test:
    Y_test.append(np.where(i == 1)[0][0])

In [78]:
len(Y_test)

276209

In [79]:
print(recall_score(Y_test, y_predicted, average='weighted'))
print(precision_score(Y_test, y_predicted, average='weighted'))
print(f1_score(Y_test, y_predicted, average='weighted'))

0.66293639961
0.657948555193
0.660131336777


In [80]:
pd.DataFrame(metrics.confusion_matrix(Y_test,y_predicted),columns = ['preparation(0)','ingredients(1)','nutrient(2)','reviews(3)','title(4)'])

,preparation(0),ingredients(1),nutrient(2),reviews(3),title(4)
0,23766,7195,971,14861,5664
1,5906,35978,816,5059,6488
2,53,152,53746,483,637
3,13984,4482,2079,31944,6189
4,6059,4512,1148,6362,37675


In [81]:
model.save('cnn-lstm-final.h5')

In [82]:
synthetic_data = ['Cook the rigatoni according to the package instructions. When there is just a few minutes left of cooking, add the baby broccolini and cook till fork-tender. Remove, drain, and set aside.','Preheat oven to 350 degrees F (175 degrees C)',' 1/2 (12 ounce) package uncooked lasagna noodles','Calories:324 kcal','This receipe exceded my expectations. I took','2 tablespoons milk','2 cups shredded mozzarella cheese, divided','This is my FAVORITE recipe in the world!','Wow! I just finished baking this lasagna','I love tofu, and I love lasagna ... I never','This was very bland. I did add more tomato','Fat: 14.5 g','Carbs: 26.2g','Protein: 24.1 g','Cholesterol: 81 mg','In a medium bowl combine tofu, eggs, salt, pep','Layer lasagna noodles with the sauce mixture','Bake in preheated oven for 25 to 35 minutes.','2 cups shredded mozzarella cheese, divided','1/2 cup grated Parmesan cheese','2 cups shredded mozzarella cheese, divided','1/4 teaspoon ground nutmeg']

In [83]:
synthetic_data[5]

'2 tablespoons milk'

## LSTM only

In [94]:
model1 = Sequential()
model1.add(LSTM(64,return_sequences=True,input_shape=(5,1)))
model1.add(LSTM(32, return_sequences=True))
model1.add(LSTM(9))
model1.add(Dense(5, activation='softmax'))

In [95]:
model1.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 5, 64)             16896     
_________________________________________________________________
lstm_11 (LSTM)               (None, 5, 32)             12416     
_________________________________________________________________
lstm_12 (LSTM)               (None, 9)                 1512      
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 50        
Total params: 30,874.0
Trainable params: 30,874
Non-trainable params: 0.0
_________________________________________________________________
None


In [96]:
history = model1.fit(X_train.reshape(X_train.shape[0],X_train.shape[1],1), y_train,
                    epochs=5, batch_size=128,
                    verbose=1, validation_split=0.3)

Train on 451140 samples, validate on 193346 samples
Epoch 1/5
451140/451140 [==============================] - 307s - loss: 0.7113 - acc: 0.7054 - val_loss: 0.6323 - val_acc: 0.7387
Epoch 2/5
451140/451140 [==============================] - 293s - loss: 0.5969 - acc: 0.7597 - val_loss: 0.5679 - val_acc: 0.7730
Epoch 3/5
451140/451140 [==============================] - 220s - loss: 0.5628 - acc: 0.7771 - val_loss: 0.5477 - val_acc: 0.7827
Epoch 4/5
451140/451140 [==============================] - 221s - loss: 0.5405 - acc: 0.7873 - val_loss: 0.5304 - val_acc: 0.7902
Epoch 5/5
451140/451140 [==============================] - 221s - loss: 0.5271 - acc: 0.7930 - val_loss: 0.5233 - val_acc: 0.7950


In [103]:
score = model1.evaluate(X_test.reshape(X_test.shape[0],X_test.shape[1],1), y_test,
                       batch_size=128, verbose=1)

276096/276209 [============================>.] - ETA: 0s

In [104]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.523923744019
Test accuracy: 0.793967611471


In [106]:
y_predicted_arr = model1.predict(X_test.reshape(X_test.shape[0],X_test.shape[1],1))

In [107]:
y_predicted = []
for i in y_predicted_arr:
    y_predicted.append(np.where(np.max(i) == i)[0][0])

In [108]:
Y_test = []
for i in y_test:
    Y_test.append(np.where(i == 1)[0][0])

In [109]:
print(recall_score(Y_test, y_predicted, average='weighted'))
print(precision_score(Y_test, y_predicted, average='weighted'))
print(f1_score(Y_test, y_predicted, average='weighted'))

0.793967611483
0.790369155034
0.791153926735


In [110]:
pd.DataFrame(metrics.confusion_matrix(Y_test,y_predicted),columns = ['preparation(0)','ingredients(1)','nutrient(2)','reviews(3)','title(4)'])

,preparation(0),ingredients(1),nutrient(2),reviews(3),title(4)
0,29296,5886,100,11907,5268
1,3294,44369,165,2204,4215
2,10,130,54921,7,3
3,8139,3136,108,44466,2829
4,3736,2937,317,2517,46249


In [120]:
model1.save('lstm-final.h5')